In [ ]:
import pickle as pkl
import numpy as np
import torch
import blosc

In [ ]:
def load(file_name):
    with open(file_name, 'rb') as f:
        loaded = pkl.load(f)
    return loaded

colors = ['red', 'green', 'blue', 'purple', 'yellow', 'grey']
types = ['door', 'key', 'ball', 'box']
actions = ["go", "pick", "up", "open", "put"]
fillers = ["to", "next", "the", "a"]
misc = ["follow_teacher"]
vocab = ['PAD'] + colors + types + actions + fillers + misc

def to_text(tokens):
    words = [vocab[token] for token in tokens if not token == 0]
    return words

def to_vocab_index(mission, pad_length=10):
    words = mission.split(" ")
    
    mission_list = [vocab.index(word) for word in words]
    if pad_length is not None:
        mission_list = mission_list + [0] * (pad_length - len(mission_list))
    if len(mission_list) > pad_length:
        raise ValueError("Mission is too long: " + mission + str(pad_length))
    return mission_list



In [ ]:
cd meta-mb-internal

In [ ]:
theirs = load('/home/olivia/Documents/Teachable/og_babyai/demos/DUMMY.pkl')

In [ ]:
len(theirs)

In [ ]:
for i in range(10):
    ours = load(f'/home/olivia/Documents/Teachable/babyai/meta-mb-internal/data/'
                'DEBUG_teacherPreActionAdvice_dense_threshS0.99_threshA0.8_lr0.0001_ent0.01_currfnone_hot_1'
                f'/buffer/traj_train_level18_idx{i}.pkl')
#     print(to_text(ours.obs[0]['instr']))
    for j in range(len(ours.obs)):
        print(to_text(ours.obs[j]['instr']))

In [ ]:
# loop through
theirs = load('/home/olivia/Documents/Teachable/og_babyai/demos/DUMMY.pkl')

for i, their_traj in enumerate(theirs):
    print(i)
    ours = load(f'/home/olivia/Documents/Teachable/babyai/meta-mb-internal/data/'
                'DEBUG_teacherPreActionAdvice_dense_threshS0.99_threshA0.8_lr0.0001_ent0.01_currfnone_hot_1'
                f'/buffer/traj_train_level18_idx{i}.pkl')
    print(to_text(ours.obs[0]['instr']))
    print(ours.teacher_action[:,0])

    # actions
    their_action = np.array([t for t in their_traj[3]])
    assert np.array_equal(ours.teacher_action[:,0], their_action), (ours.teacher_action, their_action)
    
    # obs
#     print("comparing obs")
    our_obs = ours.obs
    their_obs = blosc.unpack_array(their_traj[1])
    for oo, to in zip(our_obs, their_obs):
        assert np.array_equal(oo['obs'], to), (oo['obs'].shape, to.shape)

#     print("first")
    assert np.array_equal(ours.obs[0]['obs'], their_obs[0]), (ours.obs[0]['obs'].shape, their_obs[0].shape)
    
    # instr
    their_mission_tokens = to_vocab_index(their_traj[0])
    our_mission_tokens = ours.obs[0]['instr']
    assert np.array_equal(their_mission_tokens, our_mission_tokens), (their_mission_tokens, our_mission_tokens)
    
print("all good!")

In [ ]:
for t in theirs[:10]:
    print(t[0])